<a href="https://colab.research.google.com/github/dliseabc/fastai/blob/main/Lesson_6_Exercise_owncopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''
    Author: Huyen Nguyen
    Date created: 30/06/2021
'''
!pip install -Uqq fastbook
import fastbook
from fastbook import *

In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


##Question 1##
a. Use the documentation of CollabDataLoaders to complete the code below, using 'title' as the item name and a batch size of 128) https://docs.fast.ai/collab.html#CollabDataLoaders.from_df


In [8]:
 
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=128)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


b. (Optional) Can you rearrange the ```ratings``` dataframe into this form https://pbpython.com/pandas-crosstab.html ![](https://drive.google.com/uc?export=view&id=1nTAwITd33VXvYPszak8DUqT7-3Q_ElcU)





In [9]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
users_movies = torch.randn(n_movies, n_factors)

In [12]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

In [13]:
class DotProduct(Module):
  def _init_(self, n_users, n_movies, n_factors):
    self.user_factors = Embedding(n_users, n_factors)
    self.movie_factors = Embedding(n_movies, n_factors)

  def forward(self, x):
    users = self.user_factors(x[:0])
    movies = self.movie_factors(x[:1])
    return (users * movies).sum(dim=1)

In [14]:
x, y = dls.one_batch()
x.shape

torch.Size([128, 2])

c. How sparse is this matrix? (i.e: the number of ratings divided by all possible user-movie combinations)

In [ ]:
???

## Question 2## 
a. The step below defines the model including the forward pass and creates the embedding matrices for users and movies. Please fill in the correct dimensions of the matrices. 

b. 
Can you explain what is happening in these steps? 

```
     def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)
```

c. What's the meaning of ```y_range=(0,5.5)``` and why do we want that constraint?

d. Why do we use MSELossFlat() as the loss function instead of cross entropy?


In [16]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_factors = Embedding(???, ???)
        self.movie_bias = Embedding(???, ???)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

SyntaxError: ignored

In [15]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

NameError: ignored

In [ ]:
learn.fit_one_cycle(3, 5e-3)


e. The code below counts the number of weights in your model, can you explain the result and calculate it yourself?

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

## Question 3 (Optional)##
Can you get one batch of data from dls and show the output of the model?

In [ ]:
x,y = ???
output = ???
output

User 100 has never seen movie 250, can you show what the predicted rating would be?

In [ ]:
???

Can you plot the output of the model on the validation data vs the actual ratings?

In [ ]:
???

## Question 4 (Optional)##
```movie_bias``` captures the inherent popularity of the movies. Can you display the top 5 most popular titles?

In [ ]:
movie_bias = ???
idxs = movie_bias.argsort(???)[:5]
[dls.classes['title'][i] for i in idxs]

## Question 5##
a. What are the arguments user_sz and item_sz in the model below?

b. Why is the dimension of the linear layer ```nn.Linear(user_sz[1]+item_sz[1], n_act)```?

c. What happens in the forward pass?

In [ ]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

## Question 6##
Based on fastai documentation, what does this step ```embs = get_emb_sz(dls)``` do? 

In [ ]:
embs = get_emb_sz(dls)
model = CollabNN(*embs)

In [ ]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

## Collaborative filtering the fastai way ##

## Question 7##
a. What does ```n_factors``` mean?

In [ ]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(3, 5e-3, wd=0.1)

b. Can you define a model with 2 hidden layers with 50,20 nodes. 

In [ ]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=???)
learn.fit_one_cycle(3, 5e-3, wd=0.1)

c.(hard - optional) What is the number of parameters in your model? Can you explain the result. 
Note: the RELU layers do not have any weights. 
The BatchNorm layers ```BatchNorm1d(n)``` has 2*n weights https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html

In [ ]:
learn.model

In [ ]:
count_parameters(learn.model)

## Question 8 (hard/optional but you'll learn a great deal from doing it)
a. If we want to turn this regression task into a classification task using the CollabNN model above, what changes do we have to make?

b. Can you implement it yourself? What do you observe about the training of this new model? 